In [1]:
import os
os.chdir('/workspace/my_cyclegan')

In [ ]:
!python train.py --dataroot /all_data/hdd/un_depth/semi/sample\
--gpu_ids 1,2\
--upsampling_type transpose\
--name obtain_stable_transpose\
--continue_train\
--load_epoch 2\
--gan_mode lsgan\
--use_second_cycle\
--num_iter_gen 3\
--l_depth_A_begin 50\
--l_depth_A_end 50\
--l_depth_B_begin 50\
--l_depth_B_end 50\
--l_depth_max_iter 30000\
--l_cycle_A_begin 30.0\
--l_cycle_A_end 50.0\
--l_cycle_B_begin 30.0\
--l_cycle_B_end 50.0\
--l_identity 0.0\
--max_dataset_size 20000\
--data_shuffle\
--batch_size 10\
--lr_D 0.0001\
--lr_G 0.0005\
--img_freq 40\
--loss_freq 30\
--save_epoch_freq 1\
--n_epochs 10\
--n_epochs_decay 5

----------------- Options ---------------
               batch_size: 10                            	[default: 5]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          	[default: False]
                crop_size: 256                           
             data_shuffle: True                          	[default: False]
                 dataroot: /all_data/hdd/un_depth/semi/sample
             dataset_mode: semi_cycle                    
            deterministic: False                         
         disc_for_normals: False                         
                  dropout: False                         
              epoch_count: 1                             
                 gan_mode: lsgan                         
                  gpu_ids: 1,2                           	[default: 0,1]
                 img_freq: 40                            	[default: 1]
         

In [5]:
!python train.py --dataroot /all_data/hdd/un_depth/semi/sample\
--gpu_ids 0,1\
--name test\
--use_second_cycle\
--use_mean_matching\
--gan_mode lsgan\
--upsampling_type upconv\
--n_blocks 9\
--n_downsampling 2\
--ngf_depth 32\
--ngf_img 32\
--ndf 64\
--netD n_layers\
--n_layers_D 3\
--l_depth_A_begin 5\
--l_depth_A_end 0\
--l_depth_B_begin 5\
--l_depth_B_end 0\
--l_depth_max_iter 5000\
--l_cycle_A_begin 10.0\
--l_cycle_A_end 10.0\
--l_cycle_B_begin 10.0\
--l_cycle_B_end 10.0\
--l_identity 0\
--max_dataset_size 16\
--data_shuffle\
--batch_size 2\
--lr_D 0.0002\
--lr_G 0.0001\
--num_iter_gen 1\
--norm instance\
--init_type normal\
--beta1 0.5\
--img_freq 1\
--loss_freq 1\
--save_epoch_freq 1\
--n_epochs 1\
--n_epochs_decay 1

----------------- Options ---------------
               batch_size: 2                             	[default: 5]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
             data_shuffle: True                          	[default: False]
                 dataroot: /all_data/hdd/un_depth/semi/sample
             dataset_mode: semi_cycle                    
            deterministic: False                         
         disc_for_normals: False                         
                  dropout: False                         
              epoch_count: 1                             
                 gan_mode: lsgan                         
                  gpu_ids: 0,1                           	[default: 2]
                 img_freq: 1                             
                init_type: normal        

Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL is incompatible with libgomp.so.1 library.
	Try to import numpy first or set the threading layer accordingly. Set MKL_SERVICE_FORCE_INTEL to force it.


In [1]:
import numpy as np
from glob import glob
import imageio
from tqdm import tqdm
import os

In [2]:
def get_normal(depth):
    norm = np.zeros((2, depth.shape[0], depth.shape[1]))
    dzdx = np.gradient(depth, 1, axis=0)
    dzdy = np.gradient(depth, 1, axis=1)
    norm[ 0, :, :] = -dzdx
    norm[ 1, :, :] = -dzdy
    n = np.linalg.norm(norm, axis = 0, ord=2, keepdims=True)
    norm = norm/(n + 1e-15)
    return norm

In [2]:
path = '/all_data/hdd/un_depth/results/max/check_tanh/A2B/'
Depths = glob(os.path.join(path, 'depth/*'))

In [4]:
for d in tqdm(Depths):
    norm = get_normal(imageio.imread(d))
    name = os.path.basename(d).split('.')[0]
    np.save(os.path.join(path,'normal', name+'.npy'), norm)

100%|██████████| 5000/5000 [01:30<00:00, 55.13it/s]


In [5]:
path = '/all_data/hdd/un_depth/semi/sample/testB/'
Depths = glob(os.path.join(path, 'depth/*'))

In [6]:
for d in tqdm(Depths):
    norm = get_normal(imageio.imread(d))
    name = os.path.basename(d).split('.')[0]
    np.save(os.path.join(path,'normal', name+'.npy'), norm)

100%|██████████| 5000/5000 [01:28<00:00, 56.75it/s]


In [6]:
path = '/all_data/hdd/un_depth/semi/sample/trainA/first/'
Depths = glob(os.path.join(path, 'depth/*'))

In [7]:
len(Depths)

5000

In [8]:
for d in tqdm(Depths):
    norm = get_normal(imageio.imread(d))
    name = os.path.basename(d).split('.')[0]
    np.save(os.path.join(path,'normal', name+'.npy'), norm)

100%|██████████| 5000/5000 [01:35<00:00, 52.26it/s]


In [15]:
path = '/all_data/hdd/un_depth/semi/sample/trainA/second/'
Depths = glob(os.path.join(path, 'depth/*'))

In [16]:
for d in tqdm(Depths):
    norm = get_normal(imageio.imread(d))
    name = os.path.basename(d).split('.')[0]
    np.save(os.path.join(path,'normal', name+'.npy'), norm)

100%|██████████| 5000/5000 [01:47<00:00, 46.43it/s]


In [3]:
path = '/all_data/hdd/un_depth/results/max/ugatit/A2B/'
Depths = glob(os.path.join(path, 'depth/*'))

In [4]:
for d in tqdm(Depths):
    norm = get_normal(imageio.imread(d))
    name = os.path.basename(d).split('.')[0]
    np.save(os.path.join(path,'normal', name+'.npy'), norm)

100%|██████████| 5000/5000 [06:41<00:00, 12.44it/s]  


In [6]:
path = '/all_data/hdd/un_depth/semi/sample/trainB/first/'
Depths = glob(os.path.join(path, 'depth/*'))

In [7]:
for d in tqdm(Depths):
    norm = get_normal(imageio.imread(d))
    name = os.path.basename(d).split('.')[0]
    np.save(os.path.join(path,'normal', name+'.npy'), norm)

100%|██████████| 5000/5000 [04:43<00:00, 17.66it/s]  


In [8]:
path = '/all_data/hdd/un_depth/semi/sample/trainB/second/'
Depths = glob(os.path.join(path, 'depth/*'))

In [9]:
for d in tqdm(Depths):
    norm = get_normal(imageio.imread(d))
    name = os.path.basename(d).split('.')[0]
    np.save(os.path.join(path,'normal', name+'.npy'), norm)

100%|██████████| 5000/5000 [05:54<00:00, 14.11it/s]  


In [ ]:
/all_data/hdd/un_depth/semi/sample/trainB/second/